## SWIPE Slim
A step-by-step guide through the SWIPE [4] algorithm in a more efficient implementation. This notebook is part of **libf0 -  A Python Library for F0-Estimation in Music Recordings**.

[4] Arturo Camacho and John G. Harris. A sawtooth waveform inspired pitch estimator for speech and music. The Journal of the Acoustical Society of America, vol. 124, no. 3, pp. 1638–1652, Sep. 2008.

In [ ]:
import numpy as np
import librosa
from scipy.interpolate import interp1d

import sys
sys.path.append('../')
from libf0 import parabolic_interpolation

import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
# load demo audio (a throat microphone recording of a soprano singer)
fn_wav = "../data/DCS_LI_QuartetB_Take03_S1_LRX_excerpt.wav"
x, Fs = librosa.load(fn_wav, sr=22050)
ipd.display(ipd.Audio(x, rate=Fs, normalize=True)) # audio playback

# set all parameters for YIN
N = 2048 # frame size of the algorithm in samples
H = 256 # hop size of the algorithm in samples
F_min = 55.0 # minimum frequency of interest in Hz
F_max = 1760.0 # maximum frequency of interest in Hz
R = 10
strength_threshold = 0

In the following, we visualize the SWIPE slim algorithm step-by-step. First, let's calculate the time and frequency axes.

In [ ]:
t = np.arange(0, len(x), H) / Fs  # time axis
F_coef_log = np.arange(0, np.log2(Fs/2/F_min), R/1200)
F_coef_log_hz = F_min * 2 ** F_coef_log  # pitch candidates

The next step is the pre-computation of kernels for each pitch candidate in the range of `F_min` to `F_max`.

**TODO:**
* nicely explain and visualize the kernel(s)

In [ ]:
def prime_and_one(upto=1000000):
    """
    Returns a set of prime numbers, adapted from http://rebrained.com/?p=458

    Parameters
    ----------
    upto : int
        Find prime numbers up to this number

    Returns
    -------
    A set of prime numbers including 1 & 2
    """
    primes = np.arange(3, upto+1, 2)
    isprime = np.ones((upto-1)//2, dtype=np.bool8)
    for factor in primes[:int(np.sqrt(upto))//2]:
        if isprime[(factor-2)//2]:
            isprime[(factor*3-2)//2::factor] = 0
    return np.concatenate((np.array([1, 2]), primes[isprime]))

def compute_kernel(f, F_coef_log_hz):
    """
    Compute a SWIPE' kernel.

    Parameters
    ----------
    f : float
        Frequency in Hz
    F_coef_log_hz :
        Logarithmic frequency axis in Hz

    Returns
    -------
    k : ndarray
        Kernel
    """
    k = np.zeros(len(F_coef_log_hz))
    n_harmonics = np.floor(F_coef_log_hz[-1] / f).astype(np.int32)
    prime_numbers = prime_and_one(100)[:n_harmonics]  # only consider prime harmonics for kernel peaks

    ratio = F_coef_log_hz / f

    # loop through all prime harmonics
    for p in prime_numbers:
        a = np.abs(ratio - p)  # normalized distance between harmonic and current pitch candidate
        main_peak_bins = a < 0.25
        k[main_peak_bins] = np.cos(np.dot(np.array(2 * np.pi).reshape(-1, 1),
                                          ratio[main_peak_bins].reshape(1, -1))).flatten()
        valley_bins = np.logical_and(0.25 < a, a < 0.75)
        k[valley_bins] += np.cos(np.dot(np.array(2 * np.pi).reshape(-1, 1),
                                        ratio[valley_bins].reshape(1, -1))).flatten() / 2

    # Apply decay
    k = np.multiply(k, np.sqrt(1.0 / F_coef_log_hz))

    # K+-normalize kernel
    k = k / np.linalg.norm(k[k > 0])

    return k

# pre-compute kernels, one kernel for each pitch candidate in range [F_min : F_max]
F_min_idx = np.argmin(np.abs(F_coef_log_hz - F_min))
F_max_idx = np.argmin(np.abs(F_coef_log_hz - F_max))
B = F_max_idx - F_min_idx  # Number of pitch candidates
kernels = np.zeros((B, len(F_coef_log_hz)))
for i, f in enumerate(F_coef_log_hz[F_min_idx:F_max_idx]):
    kernels[i, :] = compute_kernel(f, F_coef_log_hz)

plt.figure()
plt.plot(kernels[0])
plt.show()

Then we calculate the optimal window length for each kernel/candidate.

**TODO:**
* show matching between Hann window in frequency domain and kernels depending on frequency
* optimal window size is $N=8/f$, show how this is quantized of the window sizes to avoid excessive STFTs

In [ ]:
L_opt = np.log2(Fs * 8 / np.array([F_min, F_max]))  # exponents for optimal window sizes 2^L, see paper Section II.G
L_rnd = np.arange(np.round(L_opt[1]), np.round(L_opt[0])+1).astype(np.int32)  # range of rounded exponents
N_pow2 = 2 ** L_rnd  # Compute rounded power-2 windows sizes
# Quantization error between optimal window size (see paper Section II.G) and rounded power-2 windows size
# Using only the largest N here, since errors for other N can be derived from err by subtracting exponent (cyclic)
err = np.abs(np.log2(8 * Fs / F_coef_log_hz[F_min_idx:F_max_idx]) - np.log2(np.max(N_pow2)))

Then we calculate the pitch strength for each window size and kernel, similar to the original SWIPE.

**TODO:**
* visualize `S_N`: the spectrum correlated with a specific kernel
* in `S`, `S_N` is weighted by `mu`, which is the error induced by the quantization of window lengths

In [ ]:
S = np.zeros((B, len(t)))  # "pitch-strength" matrix

# loop through all window sizes
for octave, N in enumerate(N_pow2):
    # Compute STFT
    x_pad = np.pad(x, (0, N))  # to avoid problems during time axis interpolation
    H = N // 2
    X = librosa.stft(x_pad, n_fft=N, hop_length=H, win_length=N, window='hann', pad_mode='constant', center=True)
    Y = np.abs(X)
    T_coef_lin_s = np.arange(0, X.shape[1]) * H / Fs
    F_coef_lin_hz = np.arange(N // 2 + 1) * Fs / N

    # Resample to log-frequency axis
    compute_Y_log = interp1d(F_coef_lin_hz, Y, kind='cubic', axis=0)
    Y_log = compute_Y_log(F_coef_log_hz)

    # Normalize magnitudes
    Y_log /= np.sqrt(np.sum(Y_log ** 2, axis=0)) + np.finfo(float).eps

    # Correlate kernels with log-spectrum for pitch candidates where N is optimal
    S_N = np.matmul(kernels, Y_log)

    # Resample time axis
    compute_S_N_res = interp1d(T_coef_lin_s, S_N, kind='linear', axis=1)
    S_N_res = compute_S_N_res(t)

    # Weight pitch strength according to quantization error
    candidates = (err > octave - 1) & (err < octave + 1)  # consider pitches +/- 1 octave from current window
    mu = 1 - np.abs(err[candidates] - octave)

    S[candidates, :] += np.multiply(mu.reshape(-1, 1), S_N_res[candidates, :])

The F0 estimate is now the maximum in S. The estimate can be refined with parabolic interpolation.

**TODO:**
* Show how parabolic interpolation improves the estimate (cf. YIN)

In [ ]:
max_indices = np.argmax(S, axis=0)
conf = np.max(S, axis=0)

# Parabolic Interpolation of pitch estimates for refinement
time_idx = np.arange(S.shape[1])
indeces_shift, _ = parabolic_interpolation(S[max_indices-1, time_idx],
                                           S[max_indices, time_idx],
                                           S[max_indices+1, time_idx])
compute_f0_log = interp1d(np.arange(len(F_coef_log)), F_coef_log, kind='linear')
f0_hz = F_min * 2 ** compute_f0_log(max_indices+indeces_shift)

# Thresholding
f0_hz[conf < strength_threshold] = 0  # discard estimates where confidence is low

# Visualize F0-Trajectory and Aperiodicity
plt.figure(figsize=(9, 6))
plt.plot(t, f0_hz, linestyle='', color='k', marker='.')
plt.ylim((0, 600))
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')

plt.figure(figsize=(9, 6))
plt.plot(t, conf, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Confidence')